In [1]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
from get_CO2 import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 1000 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 101
piter = 50
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_CO2 = 0.0001

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}

In [4]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}

In [5]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [6]:
def neg_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return -euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [7]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT, GET_CO2):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) #Need to +1 because data has CO2 observations on top of other state observations. 
    obs_model_no_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1])
    obs_model_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means, obs_error)
    net = SDEFlow(DEVICE, BATCH_SIZE, obs_model_no_CO2, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            CO2 = GET_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
            x_with_CO2 = torch.cat([C_PATH, CO2], -1)
            if iter <= PRETRAIN_ITER:
                l1_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(OBS_MODEL.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nx mean =', x_with_CO2.mean(-2))
                    print('\nx =', x_with_CO2)
                l1_norm.backward()
            else:
                log_lik = neg_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                ELBO = log_prob.mean() + log_lik.mean() - obs_model_CO2(x_with_CO2) #obs_model_CO2(x_with_CO2) is obs log likelihood.
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nx mean =', x_with_CO2.mean(-2))
                    print('\nx =', x_with_CO2)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [8]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/101 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000045771.4125. Best norm loss value is: 457714.125.

x mean = tensor([[0.6242, 0.6852, 0.9600, 0.0036],
        [0.6231, 0.6840, 0.9512, 0.0036]], grad_fn=<MeanBackward1>)

x = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.7783e-01, 8.2224e-01, 7.0917e-01, 2.9794e-03],
         [6.4882e-01, 9.3612e-01, 1.2377e+00, 3.5834e-03],
         ...,
         [6.6774e-01, 7.5729e-01, 1.6692e+00, 2.4720e-03],
         [5.7857e-01, 6.2031e-01, 2.2509e-01, 1.8467e-03],
         [2.4031e-01, 7.5291e-01, 5.2942e-02, 2.1640e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.7783e-01, 4.3750e-01, 8.2553e-01, 1.6551e-03],
         [8.5155e-01, 3.6541e-01, 6.4818e-01, 1.4340e-03],
         ...,
         [5.5768e-01, 6.2261e-01, 4.7706e-01, 1.9255e-03],
         [5.4828e-01, 6.9078e-01, 2.5819e-01, 2.0561e-03],
         [8.0874e-01, 5.1406e-01, 8.4825e-03, 1.4829e-03]]],
       grad_fn=<CatBackward>)



Train Diffusion:  10%|▉         | 10/101 [00:54<08:46,  5.78s/it]

Moving average norm loss at 10 iterations is: 438484.753125. Best norm loss value is: 424727.46875.

x mean = tensor([[3.6024e+00, 5.4072e-01, 3.9403e-01, 2.7899e-03],
        [3.6744e+00, 5.3991e-01, 3.9417e-01, 2.7840e-03]],
       grad_fn=<MeanBackward1>)

x = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [3.3613e+00, 1.4523e+00, 4.4435e-01, 5.1820e-03],
         [5.5774e+00, 3.9543e+00, 4.5883e-01, 1.4499e-02],
         ...,
         [3.1077e+00, 5.1795e-01, 2.8710e-01, 1.6260e-03],
         [1.0824e+00, 4.7042e-01, 3.2622e-01, 1.4274e-03],
         [5.4496e+00, 4.5704e-01, 3.6792e-01, 1.4222e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.8173e-01, 4.1029e+00, 4.2941e-01, 1.4364e-02],
         [2.2370e+00, 1.1015e+00, 4.5402e-01, 4.0991e-03],
         ...,
         [3.2896e-01, 3.4118e-01, 3.8558e-01, 1.0703e-03],
         [3.3119e+00, 3.7791e-01, 2.8850e-01, 1.1832e-03],
         [9.8695e-01, 4.5834e-01, 1.3217e-01, 1.3399e-03]


Train Diffusion:  20%|█▉        | 20/101 [01:47<08:55,  6.61s/it]

...


Train Diffusion:  89%|████████▉ | 90/101 [06:25<00:40,  3.64s/it]

Moving average ELBO loss at 90 iterations is: 24602320.2. Best ELBO loss value is: 6713532.0.

x mean = tensor([[1.0054e+01, 7.4278e-01, 3.1068e+00, 4.4039e-03],
        [9.9682e+00, 7.3337e-01, 3.0460e+00, 4.3812e-03]],
       grad_fn=<MeanBackward1>)

x = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.2357e-02, 1.7737e-01, 1.1653e+00, 7.8175e-04],
         [1.4214e+01, 1.6529e-01, 1.5928e-01, 8.8978e-04],
         ...,
         [1.8438e+01, 2.4298e-01, 1.0877e+01, 2.2865e-03],
         [1.3522e+01, 1.7834e-01, 7.8898e+00, 1.6277e-03],
         [1.7504e+01, 1.1637e-01, 6.4683e-02, 5.7196e-04]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.0521e+00, 8.0265e+00, 3.9280e-01, 2.8165e-02],
         [3.8656e+00, 7.5114e+00, 1.6358e+00, 2.7526e-02],
         ...,
         [6.9047e-01, 1.4282e+00, 2.7277e-02, 4.2810e-03],
         [3.0870e+00, 1.6632e+00, 5.9014e-01, 4.9708e-03],
         [4.7698e-01, 1.0928e+00, 3.9260e+00, 3.5834e-03]]],
  


Train Diffusion:  99%|█████████▉| 100/101 [07:02<00:03,  3.64s/it]

Moving average ELBO loss at 100 iterations is: 4226012.725. Best ELBO loss value is: 2577469.25.

x mean = tensor([[1.1981e+01, 3.3046e-01, 7.2639e-01, 2.0524e-03],
        [1.2067e+01, 3.4331e-01, 7.4669e-01, 2.1160e-03]],
       grad_fn=<MeanBackward1>)

x = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [6.7296e+00, 1.2512e+01, 1.4049e-01, 4.3737e-02],
         [1.2830e+01, 3.9202e-01, 3.7030e-01, 1.7158e-03],
         ...,
         [1.3315e+00, 1.0386e-01, 3.1322e+00, 7.0355e-04],
         [1.7536e+01, 9.5010e-02, 7.9011e-02, 5.2612e-04],
         [2.3022e+01, 2.1460e-01, 1.8815e+00, 1.1341e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [2.3590e-01, 3.8409e-01, 3.2324e-01, 1.3876e-03],
         [4.3232e+00, 8.5896e+00, 9.7602e-02, 3.1218e-02],
         ...,
         [2.4408e+01, 4.2364e-01, 8.9717e-03, 1.6115e-03],
         [5.2042e+00, 4.1479e-01, 1.9752e+00, 1.5139e-03],
         [9.3899e-01, 9.7750e-02, 5.6895e-02, 2.9869e-04]]],


Train Diffusion: 100%|██████████| 101/101 [07:05<00:00,  4.22s/it]
